In [ ]:
def bind_forward_double_loss(
        self,
        prompt: Union[str, List[str]] = None,
        audio_length_in_s: Optional[float] = None,
        num_inference_steps: int = 10,
        guidance_scale: float = 2.5,
        learning_rate: float = 0.1,
        negative_prompt: Optional[Union[str, List[str]]] = None,
        num_waveforms_per_prompt: Optional[int] = 1,
        clip_duration: float = 2.0,
        clips_per_video: int = 5,
        num_optimization_steps: int = 1,
        optimization_starting_point: float = 0.2,
        eta: float = 0.0,
        video_paths: Union[str, List[str]] = None,
        generator: Optional[Union[torch.Generator, List[torch.Generator]]] = None,
        latents: Optional[torch.FloatTensor] = None,
        prompt_embeds: Optional[torch.FloatTensor] = None,
        negative_prompt_embeds: Optional[torch.FloatTensor] = None,
        return_dict: bool = True,
        callback: Optional[Callable[[int, int, torch.FloatTensor], None]] = None,
        callback_steps: Optional[int] = 1,
        cross_attention_kwargs: Optional[Dict[str, Any]] = None,
        output_type: Optional[str] = "np",
    ):
        # 오디오 길이(초)를 vocoder upsample factor를 이용해 spectrogram 높이로 변환
        # → latent diffusion에서 사용할 공간 차원을 설정하는 부분
        vocoder_upsample_factor = np.prod(self.vocoder.config.upsample_rates) / self.vocoder.config.sampling_rate

        if audio_length_in_s is None:
            audio_length_in_s = self.unet.config.sample_size * self.vae_scale_factor * vocoder_upsample_factor

        height = int(audio_length_in_s / vocoder_upsample_factor)

        original_waveform_length = int(audio_length_in_s * self.vocoder.config.sampling_rate)
        if height % self.vae_scale_factor != 0:
            height = int(np.ceil(height / self.vae_scale_factor)) * self.vae_scale_factor
            logger.info(
                f"Audio length in seconds {audio_length_in_s} is increased to {height * vocoder_upsample_factor} "
                f"so that it can be handled by the model. It will be cut to {audio_length_in_s} after the "
                f"denoising process."
            )

        # 입력 체크: 프롬프트, 길이, 네거티브 프롬프트 등 유효성 검사
        self.check_inputs(
            prompt,
            audio_length_in_s,
            vocoder_upsample_factor,
            callback_steps,
            negative_prompt,
            prompt_embeds,
            negative_prompt_embeds,
        )

        # 배치 크기 계산
        if prompt is not None and isinstance(prompt, str):
            batch_size = 1
        elif prompt is not None and isinstance(prompt, list):
            batch_size = len(prompt)
        else:
            batch_size = prompt_embeds.shape[0]

        device = self._execution_device

        # classifier-free guidance 사용 여부 (guidance_scale > 1이면 조건 강화)
        do_classifier_free_guidance = guidance_scale > 1.0

        # 텍스트 프롬프트를 임베딩으로 변환 (조건부/무조건부 둘 다 준비)
        prompt_embeds = self._encode_prompt(
            prompt,
            device,
            num_waveforms_per_prompt,
            do_classifier_free_guidance,
            negative_prompt,
            prompt_embeds=prompt_embeds,
            negative_prompt_embeds=negative_prompt_embeds,
        )

        # 확산 스케줄러에서 사용할 timestep 시퀀스 설정
        self.scheduler.set_timesteps(num_inference_steps, device=device)
        timesteps = self.scheduler.timesteps

        # 초기 latent z_T 샘플링 (오디오용 채널 수, 크기 등 맞춰서 생성)
        num_channels_latents = self.unet.config.in_channels
        latents = self.prepare_latents(
            batch_size * num_waveforms_per_prompt,
            num_channels_latents,
            height,
            prompt_embeds.dtype,
            device,
            generator,
            latents,
        )
        latents_dtype = latents.dtype

        # 스케줄러에서 필요로 하는 추가 옵션(ETA 등) 준비
        extra_step_kwargs = self.prepare_extra_step_kwargs(generator, eta)

        # 비디오(조건 모달리티)를 ImageBind 입력 형태로 로드 및 변환
        # → 논문에서 말하는 M2 (조건 모달리티: 비전) 준비 단계
        image_bind_video_input = load_and_transform_video_data(
            video_paths,
            device,
            clip_duration=clip_duration,
            clips_per_video=clips_per_video,
            n_samples_per_clip=2,
        )

        # ImageBind 멀티모달 임베딩 모델 로드
        # → 비디오/오디오/텍스트를 공통 의미 공간으로 투영하는 aligner 역할
        bind_model = imagebind_model.imagebind_huge(pretrained=True)
        bind_model.eval()
        bind_model.to(device)
        for p in bind_model.parameters():
            p.requires_grad = False

        # 확산 디노이징 루프
        num_warmup_steps = len(timesteps) - num_inference_steps * self.scheduler.order
        # 일정 비율의 초기 스텝은 alignment 없이 순수 diffusion만 수행 (warmup)
        num_warmup_steps_bind = int(len(timesteps) * optimization_starting_point)

        with self.progress_bar(total=num_inference_steps) as progress_bar:
            for i, t in enumerate(timesteps):
                # classifier-free guidance 사용 시, 조건/무조건 latent를 concat
                latent_model_input = torch.cat([latents] * 2) if do_classifier_free_guidance else latents

                # 현재 timestep에 맞게 latent 스케일 조정
                latent_model_input = self.scheduler.scale_model_input(latent_model_input, t)

                # UNet으로 현재 timestep의 노이즈 ε(zt, t, p) 예측
                with torch.no_grad():
                    noise_pred = self.unet(
                        latent_model_input,
                        t,
                        encoder_hidden_states=None,
                        class_labels=prompt_embeds,
                        cross_attention_kwargs=cross_attention_kwargs,
                    ).sample.to(dtype=latents_dtype)

                # classifier-free guidance: 조건부/무조건부 예측을 조합해 조건을 강화
                if do_classifier_free_guidance:
                    noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
                    noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

                # 스케줄러로 z_t → z_{t-1} 한 스텝 디노이징
                latents = self.scheduler.step(noise_pred, t, latents, **extra_step_kwargs).prev_sample

                # 이 시점의 latent를 멀티모달 정렬을 위한 최적화 변수로 사용
                latents_temp = latents.detach()
                latents_temp.requires_grad = True

                # latent 자체를 파라미터로 두고 최적화 (training-free latent aligner 구현)
                optimizer = torch.optim.Adam([latents_temp], lr=learning_rate)

                # warmup 이후부터 ImageBind 기반 guidance 적용
                if i > num_warmup_steps_bind:
                    for optim_step in range(num_optimization_steps):
                        with torch.autograd.set_detect_anomaly(True):
                            # 예측된 z_0 계산 (디노이징된 clean latent)
                            # z0 ≈ (1 / sqrt(ᾱ_t)) * (z_t - sqrt(1 - ᾱ_t) * ε)
                            x0 = 1/(self.scheduler.alphas_cumprod[t] ** 0.5) * (
                                latents_temp - (1-self.scheduler.alphas_cumprod[t])**0.5 * noise_pred
                            )

                            # z0를 VAE 디코더를 통해 mel-spectrogram으로 복원
                            x0_mel_spectrogram = self.decode_latents(x0)

                            if x0_mel_spectrogram.dim() == 4:
                                x0_mel_spectrogram = x0_mel_spectrogram.squeeze(1)

                            # mel-spectrogram → waveform (오디오 신호 복원)
                            x0_waveform = self.vocoder(x0_mel_spectrogram)

                            # 복원된 waveform을 ImageBind 오디오 입력 포맷으로 변환
                            x0_imagebind_audio_input = load_and_transform_audio_data_from_waveform(
                                x0_waveform,
                                org_sample_rate=self.vocoder.config.sampling_rate,
                                device=device,
                                target_length=204,
                                clip_duration=clip_duration,
                                clips_per_video=clips_per_video,
                            )

                            # 텍스트 프롬프트를 ImageBind 텍스트 입력으로 준비
                            if isinstance(prompt, str):
                                prompt_bind = [prompt]
                            else:
                                prompt_bind = prompt

                            inputs = {
                                # 시각 모달리티: 비디오 클립
                                ModalityType.VISION: image_bind_video_input,
                                # 청각 모달리티: 현재 생성된 오디오
                                ModalityType.AUDIO: x0_imagebind_audio_input,
                                # 텍스트 모달리티: 사용자 프롬프트
                                ModalityType.TEXT: load_and_transform_text(prompt_bind, device),
                            }

                            # ImageBind로 멀티모달 임베딩 계산
                            # → 비전/오디오/텍스트를 같은 의미 공간으로 매핑
                            embeddings = bind_model(inputs)

                            # 텍스트-오디오 정렬 손실: 1 - cosine_similarity
                            # → 생성 오디오가 프롬프트 의미와 일치하도록 유도
                            bind_loss_text_audio = 1 - F.cosine_similarity(
                                embeddings[ModalityType.TEXT],
                                embeddings[ModalityType.AUDIO],
                            )

                            # 비전-오디오 정렬 손실: 1 - cosine_similarity
                            # → 생성 오디오가 비디오 내용과 일치하도록 유도
                            bind_loss_vision_audio = 1 - F.cosine_similarity(
                                embeddings[ModalityType.VISION],
                                embeddings[ModalityType.AUDIO],
                            )

                            # 텍스트-오디오 + 비전-오디오 이중 손실
                            # → 논문에서 말하는 dual loss 형태의 멀티모달 alignment
                            bind_loss = bind_loss_text_audio + bind_loss_vision_audio

                            # 손실을 latent에 역전파하여 z_t를 조건 모달리티 쪽으로 이동
                            bind_loss.backward()
                            optimizer.step()
                            optimizer.zero_grad()

                # 업데이트된 latent를 다음 timestep의 입력으로 사용
                latents = latents_temp.detach()

                # 진행 상황 콜백
                if i == len(timesteps) - 1 or ((i + 1) > num_warmup_steps and (i + 1) % self.scheduler.order == 0):
                    progress_bar.update()
                    if callback is not None and i % callback_steps == 0:
                        callback(i, t, latents)

        # 최종 latent를 VAE 디코더로 mel-spectrogram으로 변환
        mel_spectrogram = self.decode_latents(latents)

        # mel-spectrogram → waveform (최종 오디오 생성)
        audio = self.mel_spectrogram_to_waveform(mel_spectrogram)
        audio = audio[:, :original_waveform_length]

        if output_type == "np":
            audio = audio.detach().numpy()

        if not return_dict:
            return (audio,)

        return AudioPipelineOutput(audios=audio)
